# RESEARCH NOTEBOOK --> SUPERTREND

In [2]:
from typing import Optional, Callable
from utils.os_utils import safe_read_csv
import glob
import pandas as pd
import pandas_ta as ta  # noqa: F401


def get_market_data_dfs(trading_pair: str, intervals: list, base_path: Optional[str] = None):
    if base_path is None:
        base_path = "/Users/dardonacci/Documents/work/dashboard/data/candles/"
    market_data = {}
    for file_path in glob.glob(f"{base_path}*.csv"):
        tp = file_path.split("_")[-2]
        interval = file_path.split("_")[-1].split(".")[0]
        if tp == trading_pair and interval in intervals:
            candles_df = safe_read_csv(file_path)
            candles_df["date"] = pd.to_datetime(candles_df["timestamp"], unit='ms')
            market_data[f"{tp}_{interval}"] = candles_df
    return market_data


def get_minutes_from_interval(interval: str):
    unit = interval[-1]
    quantity = int(interval[:-1])
    conversion = {"m": 1, "h": 60, "d": 1440}
    return conversion[unit] * quantity


def order_market_data_dfs(marked_data: dict):
    return sorted(marked_data.items(), key=lambda x: get_minutes_from_interval(x[0].split("_")[-1]))


def get_dataframes_merged_by_min_resolution(trading_pair, intervals, add_indicators_func: Optional[Callable] = None):
    market_data = get_market_data_dfs(trading_pair, intervals)
    ordered_data = order_market_data_dfs(market_data)
    if add_indicators_func:
        processed_data = []
        for interval, df in ordered_data:
            processed_df = add_indicators_func(df)
            processed_data.append((interval, processed_df))
    else:
        processed_data = ordered_data
    interval_suffixes = {key: f'_{key.split("_")[1]}' for key, _ in processed_data}
    merged_df = None
    for interval, df in processed_data:
        if merged_df is None:
            merged_df = df.copy()
        else:
            merged_df = pd.merge_asof(merged_df, df.add_suffix(interval_suffixes[interval]),
                                      left_on=f"timestamp", right_on=f"timestamp{interval_suffixes[interval]}",
                                      direction="backward")
    return merged_df

In [3]:
trading_pair = "WLD-USDT"
intervals = ["1h", "3m"]

candles = get_dataframes_merged_by_min_resolution(trading_pair, intervals)

In [4]:
candles.head()[["date", "date_1h"]]

,date,date_1h
0,2023-09-12 22:00:00,2023-09-12 22:00:00
1,2023-09-12 22:03:00,2023-09-12 22:00:00
2,2023-09-12 22:06:00,2023-09-12 22:00:00
3,2023-09-12 22:09:00,2023-09-12 22:00:00
4,2023-09-12 22:12:00,2023-09-12 22:00:00


In [5]:
def add_indicators(df, length=50, multiplier=3):
    df.ta.supertrend(length=length, multiplier=multiplier, append=True)
    return df

In [6]:
candles = get_dataframes_merged_by_min_resolution(trading_pair, intervals, add_indicators)

In [34]:
candles

,timestamp,open,high,low,close,volume,quote_asset_volume,n_trades,taker_buy_base_volume,taker_buy_quote_volume,...,n_trades_1h,taker_buy_base_volume_1h,taker_buy_quote_volume_1h,date_1h,SUPERT_50_3.0_1h,SUPERTd_50_3.0_1h,SUPERTl_50_3.0_1h,SUPERTs_50_3.0_1h,percentage_distance,signal
42400,1702188000000,2.6141,2.6146,2.6083,2.6136,94295,246293.5383,1472,40232,105098.6142,...,73942,2856944,7582684.1692,2023-12-10 06:00:00,2.52500053,1,2.52500053,NaN,0.00996837,1
42401,1702188180000,2.6136,2.6209,2.6136,2.6163,148183,387950.1921,1808,77465,202801.3118,...,73942,2856944,7582684.1692,2023-12-10 06:00:00,2.52500053,1,2.52500053,NaN,0.00981613,1
42402,1702188360000,2.6164,2.622,2.6164,2.6207,78129,204668.9573,1307,53206,139385.015,...,73942,2856944,7582684.1692,2023-12-10 06:00:00,2.52500053,1,2.52500053,NaN,0.01066177,1
42403,1702188540000,2.6208,2.64,2.6192,2.6307,319552,841079.2017,4210,175880,462747.592,...,73942,2856944,7582684.1692,2023-12-10 06:00:00,2.52500053,1,2.52500053,NaN,0.01074257,1
42404,1702188720000,2.6308,2.6362,2.6262,2.633,206080,542193.9254,2798,115175,303008.475,...,73942,2856944,7582684.1692,2023-12-10 06:00:00,2.52500053,1,2.52500053,NaN,0.01102061,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43195,1702331100000,2.4337,2.4351,2.4306,2.4322,20567,50042.0176,499,7777,18924.0322,...,13666,434058,1052896.976,2023-12-11 21:00:00,2.54828979,-1,NaN,2.54828979,0.01053291,0
43196,1702331280000,2.4321,2.435,2.4309,2.4343,13835,33652.4014,371,7132,17349.9628,...,13666,434058,1052896.976,2023-12-11 21:00:00,2.54828979,-1,NaN,2.54828979,0.01123065,0
43197,1702331460000,2.4341,2.4365,2.4337,2.4355,28925,70431.1559,452,9299,22645.5708,...,13666,434058,1052896.976,2023-12-11 21:00:00,2.54828979,-1,NaN,2.54828979,0.01069062,0
43198,1702331640000,2.4352,2.4379,2.4323,2.4343,37126,90419.8573,623,17003,41416.239,...,13666,434058,1052896.976,2023-12-11 21:00:00,2.54828979,-1,NaN,2.54828979,0.01013033,0


In [37]:
columns_with_supertrend_d = [col for col in candles.columns if "SUPERTd" in col]
long_conditions = []
short_conditions = []
for col in columns_with_supertrend_d:
    long_conditions.append(candles[col] == 1)
    short_conditions.append(candles[col] == -1)


In [39]:
long_conditions

[42400    True
 42401    True
 42402    True
 42403    True
 42404    True
          ... 
 43195    True
 43196    True
 43197    True
 43198    True
 43199    True
 Name: SUPERTd_50_3.0, Length: 800, dtype: bool,
 42400     True
 42401     True
 42402     True
 42403     True
 42404     True
          ...  
 43195    False
 43196    False
 43197    False
 43198    False
 43199    False
 Name: SUPERTd_50_3.0_1h, Length: 800, dtype: bool]

In [42]:
columns_with_supertrend = [col for col in candles.columns if "SUPERT" in col]

# Conditions for long and short signals
long_condition = candles[columns_with_supertrend].apply(lambda x: all(item == 1 for item in x), axis=1)
short_condition = candles[columns_with_supertrend].apply(lambda x: all(item == -1 for item in x), axis=1)

# Choose side
candles['signal'] = 0
candles.loc[long_condition, 'signal'] = 1
candles.loc[short_condition, 'signal'] = -1

In [44]:
candles[candles["signal"] != 0][["date", "signal", "SUPERTd_50_3.0", "SUPERTd_50_3.0_1h"]]

,date,signal,SUPERTd_50_3.0,SUPERTd_50_3.0_1h


In [30]:
percentage_threshold = 0.03  # This is an example threshold value
second_timeframe = "_1h"

candles["percentage_distance"] = abs(candles["close"] - candles["SUPERT_50_3.0"]) / candles["close"]

candles["signal"] = 0
long_condition = (candles["SUPERTd_50_3.0"] == 1) & (candles[f"SUPERTd_50_3.0{second_timeframe}"] == 1) & (candles["percentage_distance"] < percentage_threshold)
short_condition = (candles["SUPERTd_50_3.0"] == -1) & (candles[f"SUPERTd_50_3.0{second_timeframe}"] == -1) & (candles["percentage_distance"] < percentage_threshold)

candles.loc[long_condition, "signal"] = 1
candles.loc[short_condition, "signal"] = -1

In [33]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def add_signal_trace(fig, df, row, col):
    # Add the signal line
    fig.add_trace(go.Scatter(x=df['date'], y=df['signal'],
                         mode='lines',
                         name='SuperTrend',
                         line=dict(color="white")),
              row=row, col=col)

def add_candlestick_trace(fig, df, row, col, name_suffix, timeframe_suffix=''):
    fig.add_trace(go.Candlestick(
        x=df[f'date{timeframe_suffix}'],
        open=df[f'open{timeframe_suffix}'],
        high=df[f'high{timeframe_suffix}'],
        low=df[f'low{timeframe_suffix}'],
        close=df[f'close{timeframe_suffix}'],
        name=f"OHLC_{name_suffix}"),
        row=row, col=col)

def add_supertrend_traces(fig, df, row, col, timeframe_suffix='', color_long='green', color_short='red'):
    fig.add_trace(go.Scatter(
        x=df[f'date{timeframe_suffix}'],
        y=df[f'SUPERTl_50_3.0{timeframe_suffix}'],
        mode='lines',
        name='SuperTrend Long',
        line=dict(color=color_long)),
        row=row, col=col)

    fig.add_trace(go.Scatter(
        x=df[f'date{timeframe_suffix}'],
        y=df[f'SUPERTs_50_3.0{timeframe_suffix}'],
        mode='lines',
        name='SuperTrend Short',
        line=dict(color=color_short)),
        row=row, col=col)

# Data Preparation
candles = candles.tail(800)

# Create Subplots
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.02,
                    row_heights=[0.5, 0.5, 0.2])

# Add Traces
add_candlestick_trace(fig, candles, 1, 1, '3m')
add_supertrend_traces(fig, candles, 1, 1)

add_candlestick_trace(fig, candles, 2, 1, '1h', second_timeframe)
add_supertrend_traces(fig, candles, 2, 1, second_timeframe)

add_signal_trace(fig, candles, 3, 1)

# Style Improvements
fig.update_layout(
    height=800,
    title_text="OHLC Chart with SuperTrend and Signals",
    yaxis_title='Price',
    xaxis_rangeslider_visible=False,
    template="plotly_dark",  # For a dark theme
    font=dict(size=12),
    margin=dict(l=50, r=50, b=100, t=100, pad=4)
)

# Update x-axis and grid properties
fig.update_xaxes(showline=True, linewidth=2, linecolor='grey', gridcolor='lightgrey', rangeslider_visible=False)
fig.update_yaxes(showline=True, linewidth=2, linecolor='grey', gridcolor='lightgrey')
# Update y-axis titles
fig.update_yaxes(title_text="Price (3m)", row=1, col=1)
fig.update_yaxes(title_text="Price (1H)", row=2, col=1)
fig.update_yaxes(title_text="Signal", row=3, col=1)

# Show the figure
fig.show()
